In [21]:
 
import numpy as np 
import pandas as pd 

import pandas_summary as ps
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate, KFold, GridSearchCV
from surprise import SVD, NMF, SlopeOne, KNNBasic, KNNWithMeans, KNNBaseline, CoClustering, BaselineOnly, NormalPredictor, KNNWithZScore
from surprise.model_selection import cross_validate, train_test_split, split, GridSearchCV, LeaveOneOut, KFold
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
import datetime
from operator import itemgetter
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.matrix_factorization import SVDpp, SVD


In [1]:
import os
for dirname, _, filenames in os.walk('../data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../data/anime.csv
../data/rating.csv


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
folder = 'kaggle/'
anime_df = pd.read_csv(folder+'anime.csv')
rate_df = pd.read_csv(folder+'rating.csv')
print('anime: ', anime_df.shape)
print('rating: ', rate_df.shape)
anime_df.head()

NameError: name 'pd' is not defined

In [ ]:
full_df = rate_df.merge(anime_df, how='left', left_on=['anime_id'], right_on=['anime_id'])
df = full_df[full_df['rating_x'] != -1]
df.head()

In [25]:
dfs = ps.DataFrameSummary(full_df)
print('categoricals: ', dfs.categoricals.tolist())
print('numerics: ', dfs.numerics.tolist())
dfs.summary()

categoricals:  ['name', 'genre', 'type', 'episodes']
numerics:  ['user_id', 'anime_id', 'rating_x', 'rating_y', 'members']


,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
count,315535.0,315535.0,315534.0,NaN,NaN,NaN,NaN,315535.0,315535.0
mean,1619.379723,11287.957295,6.06998,NaN,NaN,NaN,NaN,7.694847,202323.028878
std,934.088231,9227.912459,3.813635,NaN,NaN,NaN,NaN,0.663311,197284.556599
min,1.0,1.0,-1.0,NaN,NaN,NaN,NaN,2.0,43.0
25%,804.0,2890.0,5.0,NaN,NaN,NaN,NaN,7.32,57665.0
50%,1573.0,10012.0,7.0,NaN,NaN,NaN,NaN,7.71,135929.0
75%,2440.0,17831.0,9.0,NaN,NaN,NaN,NaN,8.17,286826.0
max,3231.0,34325.0,10.0,NaN,NaN,NaN,NaN,9.37,1013917.0
counts,315535,315535,315534,315535,315530,315535,315535,315535,315535
uniques,3231,6591,11,6591,2522,6,155,480,5764


In [26]:
min_anime_ratings = 250
filter_anime = df['anime_id'].value_counts() > min_anime_ratings
filter_anime = filter_anime[filter_anime].index.tolist()

min_user_ratings = 250
filter_users = df['user_id'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['anime_id'].isin(filter_anime)) & (df['user_id'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))
df_new

The original data frame shape:	(251672, 9)
The new data frame shape:	(25741, 9)


,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
302,5,6,8.0,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069
306,5,20,6.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
309,5,30,1.0,Neon Genesis Evangelion,"Action, Dementia, Drama, Mecha, Psychological,...",TV,26,8.32,461946
311,5,47,8.0,Akira,"Action, Adventure, Horror, Military, Sci-Fi, S...",Movie,1,8.15,215897
331,5,199,8.0,Sen to Chihiro no Kamikakushi,"Adventure, Drama, Supernatural",Movie,1,8.93,466254
...,...,...,...,...,...,...,...,...,...
315083,3224,18679,7.0,Kill la Kill,"Action, Comedy, School, Super Power",TV,24,8.23,508118
315089,3224,19815,8.0,No Game No Life,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",TV,12,8.47,602291
315093,3224,20853,7.0,Hitsugi no Chaika,"Action, Adventure, Comedy, Fantasy, Romance, S...",TV,12,7.47,164510
315096,3224,22319,8.0,Tokyo Ghoul,"Action, Drama, Horror, Mystery, Psychological,...",TV,12,8.07,618056


In [27]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_new[['user_id', 'anime_id', 'rating_x']], reader)
data

In [33]:
df_new[['user_id', 'anime_id', 'rating_x']]

,user_id,anime_id,rating_x
302,5,6,8.0
306,5,20,6.0
309,5,30,1.0
311,5,47,8.0
331,5,199,8.0
...,...,...,...
315083,3224,18679,7.0
315089,3224,19815,8.0
315093,3224,20853,7.0
315096,3224,22319,8.0


In [28]:
algo = SVD(biased=False)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)

{'test_rmse': array([1.12955534, 1.10037322, 1.12917539, 1.1270707 , 1.10555223]),
 'fit_time': (0.28948974609375,
  0.2591674327850342,
  0.2838401794433594,
  0.2886464595794678,
  0.24888849258422852),
 'test_time': (0.0282137393951416,
  0.027065753936767578,
  0.14107155799865723,
  0.07629561424255371,
  0.03567647933959961)}

In [29]:
def get_top_n(predictions, n=10, minV = 0):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        top_n[uid] = [y for y in top_n[uid] if y[1] >= minV]

    return top_n

def HitRate(topNPredicted, leftOutPredictions):
  hits = 0
  total = 0

  # For each left-out rating
  for leftOut in leftOutPredictions:

    userID = leftOut[0]
    leftOutMovieID = leftOut[1]
    # Is it in the predicted top n for this user?
    hit = False

    for movieID, predictedRating in topNPredicted[userID]:
      if leftOutMovieID == movieID:
        hit = True
        break
    if (hit) :
      hits += 1

    total += 1

  # Compute overall precision
  print("hits: ", hits, "total : ", total)
  return hits/total


def AverageReciprocalHitRank(topNPredicted, leftOutPredictions):
    summ = 0
    total = 0

  # For each left-out rating
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        # Is it in the predicted top n for this user?
        hitRank = 0
        rank = 0

        for movieID, predictedRating in topNPredicted[userID]:
            rank = rank + 1
            if leftOutMovieID == movieID:
                hitRank = rank
                break
        if (hitRank>0) :
          summ += 1.0/hitRank

        total += 1
    return summ / total


def CumulativeHitRate(topNPredicted, leftOutPredictions, cutoff):
    hits = 0
    total = 0

  # For each left-out rating
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        if (leftOut[2] >= cutoff):
            # Is it in the predicted top n for this user?
            hit = False

            for movieID, predictedRating in topNPredicted[userID]:
                if leftOutMovieID == movieID:
                    hit = True
                    break
            if (hit) :
              hits += 1

            total += 1
            # Compute overall precision
    return hits/total

def RatingHitRate(topNPredicted, leftOutPredictions):
    hits = defaultdict(float)
    total = defaultdict(float)

  # For each left-out rating
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
    # Is it in the predicted top n for this user?
        hit = False

        for movieID, predictedRating in topNPredicted[userID]:
          if leftOutMovieID == movieID:
            hit = True
            break
        if (hit) :
            hits[leftOut[2]] += 1
        total[leftOut[2]] += 1
    for rating in sorted(hits.keys()):
        print(rating, hits[rating] / total[rating])


def user_coverage(topNPredicted, minRating):
    user_coverage = 0
    for userPrediction in topNPredicted.items():
        for movieRating in userPrediction[1]:
            if movieRating[1]>minRating:
                user_coverage+=1
                break
    print("user coverage ", user_coverage/len(topNPredicted))


def get_hitrate_results(algo, train_loocv, test_loocv, topN, minVal):
  algo.fit(train_loocv)
  left_out_predictions = algo.test(test_loocv)
  loocv_anti_testset = train_loocv.build_anti_testset()
  all_predictions = algo.test(loocv_anti_testset)
  top_n_predicted = get_top_n(all_predictions, topN, minVal)
  RatingHitRate(top_n_predicted, left_out_predictions)
  hitrate = HitRate(top_n_predicted, left_out_predictions)

  accuracy.mae(left_out_predictions)
  accuracy.rmse(left_out_predictions)

  print(f'HitRate: {hitrate}')
  hitrate = CumulativeHitRate(top_n_predicted, left_out_predictions, 4)
  print(f'HitRateCumul: {hitrate}')
  print("Average Reciprocal Hit Rank: ", AverageReciprocalHitRank(top_n_predicted, left_out_predictions))
  user_coverage(top_n_predicted, 4)
  return all_predictions



def hyperTune(param_grid, data, modelTarget, targetedMetric, cv = KFold(n_splits=5, random_state=777)):
    print("starting hyper tune for "+ str(modelTarget), "with" ,param_grid )
    gs = GridSearchCV(modelTarget, param_grid, measures=["rmse","mae"], cv=cv)
    gs.fit(data)
    print("RMSE score", gs.best_score['rmse'], " with ", gs.best_params)
    print("mae score", gs.best_score['mae'], " with ", gs.best_params)

    return gs.best_score[targetedMetric], gs.best_params[targetedMetric], modelTarget


In [30]:
cv = KFold(n_splits=5, random_state=777)
cvSlope = KFold(n_splits=2, random_state=777)
targets = [
           {"name": "Co-clustering", "param_grid": {"n_epochs": [20, 30], "n_cltr_u": [3, 6], "n_cltr_i": [3, 6]},"verbose": [False], "model": CoClustering, "CV": cv},
           {"name":"Slope one", "param_grid": {}, "verbose": [False], "model": SlopeOne, "CV":cvSlope},
           {"name":"knn basic","param_grid":{"sim_options": {"name": ["msd"], "min_support":[2,5,10]}, "k": [30,40], "min_k": [1,10], "verbose": [False], "user_based": [True, False]},"model":KNNBasic, "CV":cv},
           {"name":"knn means","param_grid":{"sim_options": {"name": ["msd"], "min_support":[2,5,10]}, "k": [30,40], "min_k": [1,10], "verbose": [False], "user_based": [True, False]},"model":KNNWithMeans, "CV":cv},
           {"name":"knn Zscore","param_grid":{"sim_options": {"name": ["msd"], "min_support":[2,5,10]}, "k": [30,40], "min_k": [1,10], "verbose": [False], "user_based": [True, False]},"model":KNNWithZScore, "CV":cv},
           {"name":"Random Base", "param_grid":{}, "verbose": [False], "model":BaselineOnly, "CV":cv},
           {"name":"Baseline ALS", "param_grid":{"bsl_options": {"n_epochs": [5, 10], "reg_u": [5, 10, 20], "reg_i": [5, 10, 20], "method": ["als"]}}, "verbose": [False], "model":BaselineOnly, "CV":cv},
           {"name":"Baseline SGD", "param_grid":{"bsl_options": {"n_epochs": [5, 10],  "learning_rate": [0.005, 0.0005], "reg": [0.02, 0.01], "method": ["sgd"]}}, "verbose": [False], "model":BaselineOnly, "CV":cv},
           {"name":"Matrix factorization based", "param_grid":{"lr_all":[0.007, 0.01, 0.15]}, "verbose": [False], "model":SVD, "CV":cv},
           {"name":"Matrix factorization based + implicit rating", "param_grid":{"lr_all":[0.007, 0.01, 0.15]}, "verbose": [False], "model":SVDpp, "CV":cv}
]
# knns.KNNWithMeans
targetedMetric = "mae"
bestScoringModel = [np.inf, None, None]


LOOCV = LeaveOneOut(n_splits=1, random_state=777)
train_loocv, test_loocv  = LOOCV.split(data).__next__()
for target in targets:
    print("****** doing ", target)
    np.random.seed(40)
    scoringModel, modelParam, model = hyperTune( target["param_grid"], data, target["model"], targetedMetric, target["CV"])
    print("best mae for ", target["name"], ":", scoringModel, " with parameters :", target["param_grid"])
    if scoringModel < bestScoringModel[0]:
        print("found new best tuned model !")
        bestScoringModel[0] = scoringModel
        bestScoringModel[1] = modelParam
        bestScoringModel[2] = model

    get_hitrate_results(bestScoringModel[2](), train_loocv, test_loocv, 10, 2)
    print("************************")


get_hitrate_results(bestScoringModel[2](), train_loocv, test_loocv, 10, 2)

****** doing  {'name': 'Co-clustering', 'param_grid': {'n_epochs': [20, 30], 'n_cltr_u': [3, 6], 'n_cltr_i': [3, 6]}, 'verbose': [False], 'model': <class 'surprise.prediction_algorithms.co_clustering.CoClustering'>, 'CV': <surprise.model_selection.split.KFold object at 0x7f148b09f2b0>}
starting hyper tune for <class 'surprise.prediction_algorithms.co_clustering.CoClustering'> with {'n_epochs': [20, 30], 'n_cltr_u': [3, 6], 'n_cltr_i': [3, 6]}
RMSE score 1.1408897398277023  with  {'rmse': {'n_epochs': 30, 'n_cltr_u': 3, 'n_cltr_i': 3}, 'mae': {'n_epochs': 30, 'n_cltr_u': 3, 'n_cltr_i': 3}}
mae score 0.8590600047624412  with  {'rmse': {'n_epochs': 30, 'n_cltr_u': 3, 'n_cltr_i': 3}, 'mae': {'n_epochs': 30, 'n_cltr_u': 3, 'n_cltr_i': 3}}
best mae for  Co-clustering : 0.8590600047624412  with parameters : {'n_epochs': [20, 30], 'n_cltr_u': [3, 6], 'n_cltr_i': [3, 6]}
found new best tuned model !
6.0 0.07142857142857142
7.0 0.08571428571428572
8.0 0.12307692307692308
9.0 0.16666666666666666


[Prediction(uid=5, iid=32, r_ui=8.05999451389161, est=8.283695678046357, details={'was_impossible': False}),
 Prediction(uid=5, iid=59, r_ui=8.05999451389161, est=4.973262508815436, details={'was_impossible': False}),
 Prediction(uid=5, iid=121, r_ui=8.05999451389161, est=5.665458733401274, details={'was_impossible': False}),
 Prediction(uid=5, iid=270, r_ui=8.05999451389161, est=3.4642656735578092, details={'was_impossible': False}),
 Prediction(uid=5, iid=849, r_ui=8.05999451389161, est=4.937810081040204, details={'was_impossible': False}),
 Prediction(uid=5, iid=934, r_ui=8.05999451389161, est=6.6319231360737465, details={'was_impossible': False}),
 Prediction(uid=5, iid=1575, r_ui=8.05999451389161, est=6.290281048673088, details={'was_impossible': False}),
 Prediction(uid=5, iid=1889, r_ui=8.05999451389161, est=6.606003508708586, details={'was_impossible': False}),
 Prediction(uid=5, iid=2167, r_ui=8.05999451389161, est=4.3668200651639335, details={'was_impossible': False}),
 Predi

In [70]:
def get_user_top_n(predictions, id, n=10):
 
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
      if(uid == id):
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    

    return top_n

In [96]:
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_user_top_n(predictions, 5, n=10)
if(len(top_n) == 0):
        print("Utilisateur introuvable")
else:
  for uid, user_ratings in top_n.items():
      print(uid, [iid for (iid, _) in user_ratings])
      top_n = uid, [iid for (iid, _) in user_ratings]



5 [5114, 2251, 2001, 11741, 10087, 11061, 164, 5081, 30276, 9756]
